<a href="https://colab.research.google.com/github/ITALOZDKJ/Me-tornando-um-analista-de-dados/blob/main/ambev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import of libs

In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose as sd
import pandas_datareader.data as web
import numpy as np

# Collect of negociation datas

In [ ]:
tricker = yf.Ticker('ABEV3.SA')

In [ ]:
ambev = tricker.history(interval='1d', start='2016-01-01', end='2021-10-27')
ambev_m = tricker.history(interval='1mo', start='2016-01-01', end='2021-10-27')

In [ ]:
ambev

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-01-04,15.222132,15.222132,14.775684,14.775684,13206900,0.0,0
2016-01-05,14.810028,15.041838,14.689831,15.007495,10774200,0.0,0
2016-01-06,14.904468,15.007494,14.767100,14.861540,7739100,0.0,0
2016-01-07,14.741343,14.870125,14.466606,14.466606,15316400,0.0,0
2016-01-08,14.535292,14.767102,14.535292,14.655489,10684000,0.0,0
...,...,...,...,...,...,...,...
2021-10-20,15.300000,15.580000,15.210000,15.410000,19273600,0.0,0
2021-10-21,15.000000,15.280000,14.850000,15.140000,23499300,0.0,0
2021-10-22,14.830000,15.230000,14.540000,15.000000,29599100,0.0,0


# Extraction of features

In [ ]:
ambev = tricker.history(interval='1d', start='2016-01-01', end='2021-10-27')
ambev.reset_index(inplace=True)

In [ ]:
def features_initial(x):
    x['Day'] = x['Date'].dt.day
    x['Month'] = x['Date'].dt.month
    x['Year'] = x['Date'].dt.year
    x['Rentability%'] = round((x['Close']/x['Close'].shift()) * 100 -100, 2)
    x['Rentability pure'] = round(x['Close'] - x['Close'].shift(),2)

In [ ]:
features_initial(ambev)

In [ ]:
ambev

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Day,Month,Year,Rentability%,Rentability pure
0,2016-01-04,15.222130,15.222130,14.775682,14.775682,13206900,0.0,0,4,1,2016,NaN,NaN
1,2016-01-05,14.810025,15.041834,14.689828,15.007491,10774200,0.0,0,5,1,2016,1.57,0.23
2,2016-01-06,14.904470,15.007496,14.767102,14.861542,7739100,0.0,0,6,1,2016,-0.97,-0.15
3,2016-01-07,14.741343,14.870125,14.466606,14.466606,15316400,0.0,0,7,1,2016,-2.66,-0.39
4,2016-01-08,14.535290,14.767100,14.535290,14.655487,10684000,0.0,0,8,1,2016,1.31,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,2021-10-20,15.300000,15.580000,15.210000,15.410000,19273600,0.0,0,20,10,2021,1.78,0.27
1440,2021-10-21,15.000000,15.280000,14.850000,15.140000,23499300,0.0,0,21,10,2021,-1.75,-0.27
1441,2021-10-22,14.830000,15.230000,14.540000,15.000000,29599100,0.0,0,22,10,2021,-0.92,-0.14
1442,2021-10-25,15.010000,15.440000,15.000000,15.270000,19933100,0.0,0,25,10,2021,1.80,0.27


## Creating a function to extract more datas

In [ ]:
for month in range(1,13):
        x.loc[(x['Day'] == 22) & (x['Month']==month),['Bougth']] = 1

6

In [ ]:
# Extract datas and restar index
ambev = tricker.history(interval='1d', start='2017-01-01', end='2021-10-27')
ambev.reset_index(inplace=True)
features_initial(ambev)
# Create a new dataframe
desempenho_ambev = ambev
# Create a function
def features_cash(x, y):
    x.loc[:,'Action_value'] = (y['High'] + y['Low'])/2
    year_temp = 0
    # Instance of apports
    for year in list(x['Year'].unique()):
        year_temp = x.loc[(x['Year']==year)]
        for month in list(year_temp['Month'].unique()):
            temp = x.loc[(x['Month'] == month) & (x['Year']==year)]
            day = list(temp[-1:]['Day'])
            day = day[0]
            x.loc[(x['Day'] == day) & (x['Month'] == month) & (x['Year'] == year),['Bougth']] = 1
    # Aport value
    x.loc[:,'Valor_aportado'] = x['Bougth'] * x['Action_value']
    x.fillna(0, inplace=True)
    # The sum of actions
    x['Total_actions'] = 0
    for pos,c in enumerate(x['Bougth']):
        x.loc[pos,'Total_actions'] = x.loc[:pos,'Bougth'].sum()
    
    # The sum of invest value 
    x['Total_aport'] = 0
    for pos,c in enumerate(x['Total_actions']):
        x.loc[pos,'Total_aport'] = x.loc[:pos,'Total_actions'].sum()
        
    # The sum of dividends receives with actual aports
    x['Dividend_receive'] = x['Total_actions'] * x['Dividends']
    
    # The final dividends receives
    x['Total_receive'] = 0
    for pos,c in enumerate(x['Dividend_receive']):
        x.loc[pos,'Total_receive'] = x.loc[:pos,'Dividend_receive'].sum()
    
    

In [ ]:
features_cash(desempenho_ambev, ambev)

In [ ]:
desempenho_ambev

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Day,Month,Year,Rentability%,Rentability pure,Action_value,Bougth,Valor_aportado,Total_actions,Total_aport,Dividend_receive,Total_receive
0,2017-01-02,14.540124,14.824875,14.468936,14.513428,4265400,0.0,0,2,1,2017,0.00,0.00,14.646906,0.0,0.000,0,0,0.0,0.0000
1,2017-01-03,14.682497,14.833772,14.629107,14.807076,9535500,0.0,0,3,1,2017,2.02,0.29,14.731439,0.0,0.000,0,0,0.0,0.0000
2,2017-01-04,14.824875,14.824875,14.620210,14.691398,9426500,0.0,0,4,1,2017,-0.78,-0.12,14.722542,0.0,0.000,0,0,0.0,0.0000
3,2017-01-05,14.700297,14.904961,14.664702,14.771484,11555800,0.0,0,5,1,2017,0.55,0.08,14.784832,0.0,0.000,0,0,0.0,0.0000
4,2017-01-06,14.762584,14.807076,14.557920,14.646903,7863200,0.0,0,6,1,2017,-0.84,-0.12,14.682498,0.0,0.000,0,0,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190,2021-10-20,15.300000,15.580000,15.210000,15.410000,19273600,0.0,0,20,10,2021,1.78,0.27,15.395000,0.0,0.000,57,33674,0.0,55.4965
1191,2021-10-21,15.000000,15.280000,14.850000,15.140000,23499300,0.0,0,21,10,2021,-1.75,-0.27,15.065000,0.0,0.000,57,33731,0.0,55.4965
1192,2021-10-22,14.830000,15.230000,14.540000,15.000000,29599100,0.0,0,22,10,2021,-0.92,-0.14,14.885000,0.0,0.000,57,33788,0.0,55.4965
1193,2021-10-25,15.010000,15.440000,15.000000,15.270000,19933100,0.0,0,25,10,2021,1.80,0.27,15.220000,0.0,0.000,57,33845,0.0,55.4965


### Space to tests and checks...

In [ ]:
temp = desempenho_ambev.loc[(desempenho_ambev['Month'] == 5) & (desempenho_ambev['Year']==2019)]
a = list(temp[-1:]['Day'])
a[0]

31

In [ ]:
ambev['Year'].unique()

array([2017, 2018, 2019, 2020, 2021])

In [ ]:
desempenho_ambev[desempenho_ambev['Bougth'] != 0]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Day,Month,Year,Rentability%,Rentability pure,Action_value,Bougth,Valor_aportado,Total_actions,Total_aport,Dividend_receive,Total_receive
14,2016-01-22,14.097427,14.466605,14.097427,14.466605,15564500,0.00,0,22,1,2016,3.25,0.46,14.282016,1.0,14.282016,1,1,0.0,0.0000
32,2016-02-22,16.132982,16.271314,15.942776,16.072462,14696300,0.00,0,22,2,2016,1.47,0.23,16.107045,1.0,16.107045,2,20,0.0,0.1300
53,2016-03-22,16.167568,16.202150,15.986006,16.202150,10495100,0.00,0,22,3,2016,0.05,0.01,16.094078,1.0,16.094078,3,63,0.0,0.1300
74,2016-04-22,16.305897,16.815997,16.305897,16.565269,12394900,0.00,0,22,4,2016,-0.21,-0.03,16.560947,1.0,16.560947,4,127,0.0,0.1300
116,2016-06-22,16.072458,16.098396,15.769857,15.839023,15328400,0.00,0,22,6,2016,-1.40,-0.22,15.934126,1.0,15.934126,5,296,0.0,0.1300
138,2016-07-22,16.746654,16.938143,16.633501,16.877214,8907900,0.00,0,22,7,2016,0.99,0.17,16.785822,1.0,16.785822,6,407,0.0,0.7800
159,2016-08-22,17.277604,17.312420,16.999074,17.051298,10928000,0.00,0,22,8,2016,-1.31,-0.23,17.155747,1.0,17.155747,7,534,0.0,0.7800
181,2016-09-22,17.129636,17.164451,17.007779,17.120932,10250100,0.00,0,22,9,2016,1.08,0.18,17.086115,1.0,17.086115,8,689,0.0,0.7800
221,2016-11-22,15.485582,15.564590,15.029092,15.072985,12788200,0.00,0,22,11,2016,-1.60,-0.25,15.296841,1.0,15.296841,9,1010,0.0,2.0600
243,2016-12-22,14.433341,14.451139,14.050707,14.077402,13056700,0.22,0,22,12,2016,-1.86,-0.27,14.250923,1.0,14.250923,10,1209,2.2,4.2600


In [ ]:
year_temp = 0
meses = []
for year in list(desempenho_ambev['Year'].unique()):
        year_temp = desempenho_ambev.loc[(desempenho_ambev['Year']==year)]
        for month in list(year_temp['Month'].unique()):
            meses.append(month)
len(meses)

70

In [ ]:
desempenho_ambev['Valor_aportado'].sum() 

83489.35075564646

In [ ]:
desempenho_ambev['Rentability pure'].sum() + 

0.25000000000000044

In [ ]:
a = pd.Series([1,2,3,4,5,6,7,8])

In [ ]:
a[3:]

3    4
4    5
5    6
6    7
7    8
dtype: int64

In [ ]:
loc[(x['Day'] == 22) & (x['Month']==month),['Bougth']]

In [ ]:
ambev[ambev.index == 1]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Day,Month,Year,Rentability%,Rentability pure,Action_value
1,2016-01-05,14.810026,15.041836,14.68983,15.007493,10774200,0.0,0,5,1,2016,1.57,0.23,14.865833
